<a href="https://colab.research.google.com/github/Sunnn-y/NaturalLanguageProcessing/blob/main/20231019_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5(Naive_Bayse).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayse 감정분석

- 참고<br/>
<img src="https://drive.google.com/uc?id=1I7cu4UoWZpDwXgkym1SvijVewoVPuFZr" height = 300 width = 500>
<img src="https://drive.google.com/uc?id=1q-DdZoqt0fTpb7mpoFnd6xijM1pRLY34" height = 500 width = 500>


## 실습 1
- "happy weekend" 라는 단어가 포함된 문장이 긍정일 확률과 부정일 확률은 각각 얼마일까요?

### 모델링 날코딩 (진행중)

In [50]:
text = [("I love you", "P"),
        ("love happy weekend", "P"),
        ("bore work job", "N"),
        ("I hate you", "N"),
        ("bore weekend", "N"),
        ("happy together", "P")]

In [11]:
word = []
for doc in text:
  word.append(doc[0].split(' '))

word

[['I', 'love', 'you'],
 ['love', 'happy', 'weekend'],
 ['bore', 'work', 'job'],
 ['I', 'hate', 'you'],
 ['bore', 'weekend'],
 ['happy', 'together']]

In [52]:
# 고유 토큰 추출 후 인덱스 붙이기
docu = set() # 중복값 제거를 위해 빈 세트 설정
doc_ls = []
for doc in text:
  doc_ls.append(doc[0])

for i in range(len(doc_ls)):
  token = doc_ls[i].split(' ')
  for k in token:
    docu.add(k)

docu

{'I',
 'bore',
 'happy',
 'hate',
 'job',
 'love',
 'together',
 'weekend',
 'work',
 'you'}

In [58]:
docu = list(docu)

In [ ]:
# 여기서부터 다시 하기!!
count_token = []
for doc in text:
  # print(doc[1])
  if doc[1] == "P":
#     for i in range(len(docu)):
#       count_token.append(doc[0].count(docu[i]))
# count_token

## 실습 1 - 다른 풀이(sklearn 패키지 사용)

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

text = [("I love you", "P"),
        ("love happy weekend", "P"),
        ("bore work job", "N"),
        ("I hate you", "N"),
        ("bore weekend", "N"),
        ("happy together", "P")]

In [62]:
X_train = [t[0] for t in text]
Y_train = [t[1] for t in text]
X_train

['I love you',
 'love happy weekend',
 'bore work job',
 'I hate you',
 'bore weekend',
 'happy together']

In [63]:
Y_train

['P', 'P', 'N', 'N', 'N', 'P']

- sklearn CountVectorizer<br/>
문서 집합에서 단어 토큰을 생성하고 각 단어수를 세어 BOW 인코딩 벡터를 만든다

In [64]:
count_vect = CountVectorizer() # CountVectorizer 선언
X_train_counts = count_vect.fit_transform(X_train) # fit : 학습, transform : 적용  # 토큰화 안 해도 모듈이 알아서 공백으로 토큰화해줌
model = MultinomialNB() # MultinomialNB 선언
model.fit(X_train_counts, Y_train)

MultinomialNB()

In [65]:
# label 예측
model.predict(count_vect.transform(["happy weekend"]))

array(['P'], dtype='<U1')

In [66]:
# label 예측의 확률값
model.predict_proba(count_vect.transform(["happy weekend"]))
# y label은 설정 안하면 알파벳 순으로 인덱스 매겨짐(N -> P순이니까 N=0, P=1)
# 그래서 보통 전처리 단계에서 y label을 숫자로 다 바꿔준다

array([[0.25, 0.75]])

## 실습 2
- 네이버 영화 리뷰데이터로 감정 분석

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/핀테크 데이터분석/자연어 처리/review.csv',
                 sep = '\t')
# df = df[:100]
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 토큰화 및 데이터 전처리

In [70]:
# !pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.4 MB/s eta 0:00:00


In [94]:
from konlpy.tag import Okt
tokenizer = Okt()

df['morphs'] = None
for i, row in df.iterrows():
  df['morphs'][i] = ' '.join(tokenizer.morphs(row['document']))

<ipython-input-94-eab88834c7d3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['morphs'][i] = ' '.join(tokenizer.morphs(row['document']))


In [95]:
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 .. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만...


In [96]:
import re
# 한글을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-하-ㅣ\\s]")

def preprocess_remove(text):
  text = re.sub(remove_except_ko,' ',text).strip()
  return text

df['morphs'] = df['morphs'].map(lambda x : preprocess_remove(x))
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...


In [97]:
# 불용어 처리
stop_word = ['포스터', '저작권자', '한경', '닷컴', '뉴스룸', '홈페이지', '바로', '네이버', '구독', '채널', '세요', '제공','께서', '라고', '입니다', '습니다', '한다', '에서', '이다', '에게', '으로', '이랑', '까지', '부터', '하다', '한데', '통해', '위해', '때문' ]

def preprocess(text):
  text = text.split()
  text = [i for i in text if len(i)>1]
  text = [i for i in text if i not in stop_word]
  return text

In [98]:
# 토큰화 + 토큰리스트 생성
def make_tokens(df):
  df['tokens'] = None
  for i, row in df.iterrows():
    if i%1000==0:
      print(i,'/',len(df))
    token = preprocess(df['morphs'][i])
    df['tokens'][i] = ' '.join(token)
  return df

df = make_tokens(df)

0 / 10000


<ipython-input-98-86dc58720036>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'][i] = ' '.join(token)


1000 / 10000
2000 / 10000
3000 / 10000
4000 / 10000
5000 / 10000
6000 / 10000
7000 / 10000
8000 / 10000
9000 / 10000


In [99]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 늙어 보이기만 했던 커스틴 던스트...


### 감정 분석

In [100]:
x = df.iloc[:, -1]
y = df.iloc[:, 2]

In [101]:
# 훈련데이터, 테스트데이터 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,
                                                    random_state = 2) # test_size 의 디폴트값은 0.25

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [103]:
# DTM - Document Term Matrix
dtmvector = CountVectorizer()
dtmvector.fit(x_train)
x_train_dtm = dtmvector.transform(x_train)
print(x_train_dtm.shape)

(8000, 17339)


In [105]:
# 사이킷 런에서 제공하는 나이브 베이즈 모델
model = MultinomialNB()
model.fit(x_train_dtm, y_train)

MultinomialNB()

In [106]:
# 테스트 데이터 DTM 형태로 변환
x_test_dtm = dtmvector.transform(x_test)
print(x_test_dtm.shape)

(2000, 17339)


### 정확도(acc), f1 score 구하기

In [107]:
# 정확도(accuracy) 구하기
predicted = model.predict(x_test_dtm) #테스트 데이터에 대한 예측
print("정확도: %.4f" % accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7880


In [112]:
# f1 score 구하기
from sklearn.metrics import f1_score
print(f1_score(y_test, predicted, average='macro'))

0.7878778176229508


### label 예측

In [113]:
# label 예측
model.predict(dtmvector.transform(["올해 본 영화 중 가장 최고야"])) # 0=부정, 1=긍정

array([1])

In [114]:
# label 예측의 확률값
model.predict_proba(dtmvector.transform(["올해 본 영화 중 가장 최고야"]))

array([[0.18416898, 0.81583102]])